# Get Candlestick Patterns

## Import relevant libraries

In [15]:
import os
import pandas as pd
import talib
import mplfinance as mpf
import numpy as np
import matplotlib.pyplot as plt
import datetime

## Get the data

### Parameters

In [33]:
asset_ticket = "BTCUSDT"
timestamp = "15m"
start_date = "1 Jan, 2020"
end_date = "31 Dec, 2021"

In [34]:
input_data_path = '../data'
input_data_filename = "binance" + \
                  "_" + asset_ticket + \
                  "_" + timestamp + \
                  "_from_" + datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                  "_to_" + datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y_%m_%d')
input_data_extension = ".csv"
full_path_input_data = os.path.join(input_data_path, input_data_filename + input_data_extension)

In [35]:
df = pd.read_csv(full_path_input_data)

In [36]:
df.columns

Index(['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time',
       'quote_asset_volumne', 'number_of_trades',
       'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore',
       'formatted_open_time', 'formatted_close_time', 'upper_shadow',
       'lower_shadow', 'real_body'],
      dtype='object')

## Identify Candlestick Patterns

In [37]:
candle_names = talib.get_function_groups()['Pattern Recognition']

- Inverted Hammer --> Single [Bullish]
- Hammer --> Single [Bullish]
- Bullish Engulfing (not available at ta-lib) --> Double [Bullish]
- Piercing Pattern --> Double [Bullish]
- Morning Star --> Triple [Bullish]


- Shooting Star --> Single [Bearish]
- Hanging Man --> Single [Bearish]
- Bearish Engulfing (not available at ta-lib) --> Double [Bearish]
- Dark Cloud Cover --> Double [Bearish]
- Evening Star --> Triple [Bearish]

- Engulfing [positive: Bullish / negative: Bearish]

In [38]:
candle_names = [
    'CDLINVERTEDHAMMER',
    'CDLHAMMER',
    'CDLPIERCING',
    'CDLMORNINGSTAR',
    'CDLSHOOTINGSTAR',
    'CDLHANGINGMAN',
    'CDLDARKCLOUDCOVER',
    'CDLEVENINGSTAR',
    'CDLENGULFING'
]

In [39]:
# extract OHLC 
op = df['open']
hi = df['high']
lo = df['low']
cl = df['close']

# create columns for each pattern
for candle in candle_names:
    # below is same as;
    # df["CDL3LINESTRIKE"] = talib.CDL3LINESTRIKE(op, hi, lo, cl)
    df[candle] = getattr(talib, candle)(op, hi, lo, cl)

In [40]:
df.sample(10)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,real_body,CDLINVERTEDHAMMER,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING
32106,1606788900000,19619.99,19679.30,19562.49,19634.88,639.753705,1606789799999,1.256100e+07,11500,349.889226,...,14.89,0,0,0,0,0,0,0,0,0
25973,1601265600000,10885.21,10895.00,10871.24,10871.25,399.458365,1601266499999,4.347850e+06,5146,207.747700,...,13.96,0,0,0,0,0,0,0,0,0
39979,1613897100000,56361.36,56676.70,56358.87,56513.22,636.854607,1613897999999,3.600396e+07,17864,382.105120,...,151.86,0,0,0,0,0,0,0,0,0
1963,1579603500000,8663.59,8669.54,8650.17,8659.23,366.295720,1579604399999,3.171575e+06,3280,184.269651,...,4.36,0,0,0,0,0,0,0,0,0
2436,1580029200000,8417.20,8444.30,8415.00,8415.01,502.995886,1580030099999,4.238936e+06,4260,211.995112,...,2.19,100,0,0,0,0,0,0,0,0
54970,1627419600000,38031.90,38237.00,37950.00,38230.91,399.212548,1627420499999,1.520232e+07,18287,210.786385,...,199.01,0,0,0,0,0,0,0,0,0
30516,1605354300000,15906.60,15939.99,15890.27,15925.44,559.350592,1605355199999,8.901106e+06,9509,294.249182,...,18.84,0,0,0,0,0,0,0,0,0
10102,1586960100000,6733.94,6742.65,6726.16,6739.91,463.840732,1586960999999,3.124008e+06,4369,258.321772,...,5.97,0,0,0,0,0,0,0,0,0
18227,1594294200000,9382.93,9397.00,9380.00,9394.50,357.355550,1594295099999,3.355344e+06,4835,184.719643,...,11.57,0,0,0,0,0,0,0,0,0
25601,1600930800000,10289.79,10314.92,10289.62,10308.69,359.795407,1600931699999,3.707837e+06,5607,195.665948,...,18.90,0,0,0,0,0,0,0,0,0


## Split 'CDLENGULFING' into Bullish and Bearish one

In [41]:
list_open_time_bullish_engulfing = df[df['CDLENGULFING']==100]['open_time'].to_list()
list_open_time_bearish_engulfing = df[df['CDLENGULFING']==-100]['open_time'].to_list()

In [42]:
df['CDLENGULFINGBULLISH'] = np.where(df['open_time'].isin(list_open_time_bullish_engulfing), 100, 0)
df['CDLENGULFINGBEARISH'] = np.where(df['open_time'].isin(list_open_time_bearish_engulfing), -100, 0)

### Check

In [43]:
df[df['CDLENGULFING']==100].sample(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING,CDLENGULFINGBULLISH,CDLENGULFINGBEARISH
17928,1594025100000,9183.91,9193.89,9180.38,9192.02,382.300177,1594025999999,3.511475e+06,3681,218.124040,...,0,0,0,0,0,0,0,100,100,0
51197,1624023900000,37141.89,37282.88,37105.00,37207.88,1318.592027,1624024799999,4.906444e+07,29546,493.572937,...,0,0,0,0,0,0,0,100,100,0
52368,1625077800000,34613.41,34869.14,34559.67,34853.02,669.915041,1625078699999,2.325097e+07,13147,349.915021,...,0,0,0,0,0,0,0,100,100,0
68020,1639188000000,47919.99,48227.00,47900.00,48144.15,188.318360,1639188899999,9.056778e+06,7911,96.837040,...,0,0,0,0,0,0,0,100,100,0
61388,1633219200000,47634.89,47808.19,47623.74,47705.18,425.592340,1633220099999,2.031142e+07,15249,234.860930,...,0,0,0,0,0,0,0,100,100,0


In [44]:
df[df['CDLENGULFING']==-100].sample(5)

,open_time,open,high,low,close,volume,close_time,quote_asset_volumne,number_of_trades,taker_buy_base_asset_volume,...,CDLHAMMER,CDLPIERCING,CDLMORNINGSTAR,CDLSHOOTINGSTAR,CDLHANGINGMAN,CDLDARKCLOUDCOVER,CDLEVENINGSTAR,CDLENGULFING,CDLENGULFINGBULLISH,CDLENGULFINGBEARISH
35153,1609549200000,29197.93,29202.67,29100.00,29109.66,416.155548,1609550099999,1.213657e+07,8684,158.922147,...,0,0,0,0,0,0,0,-100,0,-100
65083,1636544700000,66735.01,66783.90,66535.01,66546.80,401.524010,1636545599999,2.676404e+07,16524,215.515170,...,0,0,0,0,0,0,0,-100,0,-100
65369,1636802100000,63767.70,63804.83,63606.43,63636.30,179.708480,1636802999999,1.144823e+07,8393,66.582500,...,0,0,0,0,0,0,0,-100,0,-100
51919,1624673700000,31989.35,32033.33,31750.00,31784.34,454.064140,1624674599999,1.446144e+07,9001,231.348413,...,0,0,0,0,0,0,-100,-100,0,-100
4545,1581930900000,9819.26,9832.92,9790.00,9792.29,506.106181,1581931799999,4.965052e+06,5906,275.270495,...,0,0,0,0,0,0,0,-100,0,-100


## Export the Data

### Export parameters

In [45]:
export_path = "../data"
export_filename = "binance" + \
                  "_" + asset_ticket + \
                  "_" + timestamp + \
                  "_from_" + datetime.datetime.strptime(start_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                  "_to_" + datetime.datetime.strptime(end_date,'%d %b, %Y').strftime('%Y_%m_%d') + \
                  "_candlesticks_signals_raw"
export_extension = ".csv"
full_export_path = os.path.join(export_path, export_filename + export_extension)

### Export

In [46]:
df.to_csv(full_export_path, index=False)